In [41]:
import random

### Creacion de la Clase Vehiculo

In [42]:
class Vehiculo:
    def __init__(self, tipo_vehiculo="coche", velocidad_maxima=100):
        # Inicializa las propiedades del vehículo
        self.tipo_vehiculo = tipo_vehiculo
        self.velocidad_maxima = velocidad_maxima
        self.velocidad_actual = 0
        self.destino = None
        # El comportamiento se elige al azar entre: cauteloso, normal y agresivo
        self.comportamiento = random.choice(["cauteloso", "normal", "agresivo"])

    def acelerar(self):
        # Si la velocidad actual es menor que la máxima, el vehículo acelera
        # La cantidad de aceleración depende del comportamiento del vehículo
        if self.velocidad_actual < self.velocidad_maxima:
            incremento = (
                5 if self.comportamiento == "cauteloso" else
                15 if self.comportamiento == "agresivo" else
                10
            )
            self.velocidad_actual += incremento

    def desacelerar(self):
        # Decrementa la velocidad según el comportamiento del vehículo
        decremento = (
            5 if self.comportamiento == "agresivo" else
            15 if self.comportamiento == "cauteloso" else
            10
        )
        if self.velocidad_actual - decremento >= 0:
            self.velocidad_actual -= decremento

### Creacion de la Clase SegmentoCarretera

In [43]:
class SegmentoCarretera:
    def __init__(self, capacidad, limite_velocidad):
        # Propiedades de un segmento de carretera
        self.vehiculos = []
        self.capacidad = capacidad
        self.limite_velocidad = limite_velocidad
        self.congestion = False

    def agregar_vehiculo(self, vehiculo):
        # Agrega un vehículo al segmento si no ha alcanzado su capacidad
        if len(self.vehiculos) < self.capacidad:
            self.vehiculos.append(vehiculo)
            vehiculo.destino = self

    def flujo_vehiculos(self):
        # Gestiona el flujo de vehículos en el segmento
        for vehiculo in self.vehiculos:
            if len(self.vehiculos) > 0.8 * self.capacidad:
                # Si hay congestión, desacelera los vehículos
                self.congestion = True
                vehiculo.desacelerar()
            elif vehiculo.velocidad_actual < self.limite_velocidad and not self.congestion:
                # Si no hay congestión y la velocidad es menor al límite, acelera
                vehiculo.acelerar()

    def num_vehiculos(self):
        # Retorna el número de vehículos en el segmento
        return len(self.vehiculos)

    def mover_vehiculos_a_interseccion(self, interseccion):
        # Mueve el primer vehículo del segmento a la intersección
        if self.vehiculos:
            vehiculo = self.vehiculos.pop(0)
            interseccion.vehiculos_esperando.append(vehiculo)

### Creacion de la Clase Interseccion

In [44]:
class Interseccion:
    def __init__(self):
        # Propiedades de una intersección: semáforo y vehículos esperando
        self.vehiculos_esperando = []
        self.semaforo = random.choice(["rojo", "verde"])
        self.tiempo_semaforo = 0  # Contador para cambiar el semáforo

    def cambiar_semaforo(self):
        # Cambia el semáforo de rojo a verde o viceversa cada 10 ciclos
        if self.tiempo_semaforo >= 10:
            self.semaforo = "verde" if self.semaforo == "rojo" else "rojo"
            self.tiempo_semaforo = 0
        self.tiempo_semaforo += 1

    def manejar_interseccion(self, siguiente_segmento=None):
        # Si el semáforo está en verde y hay vehículos esperando, mueve el vehículo al siguiente segmento
        if self.semaforo == "verde" and self.vehiculos_esperando:
            vehiculo = self.vehiculos_esperando.pop(0)
            if siguiente_segmento:
                siguiente_segmento.agregar_vehiculo(vehiculo)

### Creacion de la Clase Calle

In [45]:
class Calle:
    def __init__(self, num_segmentos, capacidad_segmento, limite_velocidad):
        # Una calle tiene varios segmentos y intersecciones
        self.segmentos = [SegmentoCarretera(capacidad_segmento, limite_velocidad) for _ in range(num_segmentos)]
        self.intersecciones = [Interseccion() for _ in range(num_segmentos + 1)]

    def flujo_en_calle(self):
        # Gestiona el flujo de vehículos y semáforos en cada segmento y intersección de la calle
        for i, segmento in enumerate(self.segmentos):
            segmento.flujo_vehiculos()
            self.intersecciones[i].cambiar_semaforo()
            segmento.mover_vehiculos_a_interseccion(self.intersecciones[i])
            siguiente_segmento = self.segmentos[i + 1] if i + 1 < len(self.segmentos) else None
            self.intersecciones[i].manejar_interseccion(siguiente_segmento)

### Creacion de la Clase Vecindario

In [46]:
class Vecindario:
    def __init__(self, num_calles, num_segmentos_por_calle, capacidad_segmento, limite_velocidad):
        # Un vecindario tiene varias calles
        self.calles = [Calle(num_segmentos_por_calle, capacidad_segmento, limite_velocidad) for _ in range(num_calles)]

    def flujo_en_vecindario(self):
        # Gestiona el flujo de vehículos en cada calle del vecindario
        for calle in self.calles:
            calle.flujo_en_calle()

### Creacion de la Clase Ciudad

In [47]:
class Ciudad:
    def __init__(self, num_vecindarios, num_calles_por_vecindario, num_segmentos_por_calle, capacidad_segmento, limite_velocidad):
        # Una ciudad tiene varios vecindarios
        self.vecindarios = [
            Vecindario(num_calles_por_vecindario, num_segmentos_por_calle, capacidad_segmento, limite_velocidad)
            for _ in range(num_vecindarios)
        ]

    def flujo_en_ciudad(self):
        # Gestiona el flujo de vehículos en cada vecindario de la ciudad
        for vecindario in self.vecindarios:
            vecindario.flujo_en_vecindario()

    def introducir_vehiculos(self, num_vehiculos):
        # Introduce vehículos aleatoriamente en la ciudad
        for _ in range(num_vehiculos):
            vecindario_aleatorio = random.choice(self.vecindarios)
            calle_aleatoria = random.choice(vecindario_aleatorio.calles)
            segmento_aleatorio = random.choice(calle_aleatoria.segmentos)
            vehiculo = Vehiculo()
            segmento_aleatorio.agregar_vehiculo(vehiculo)

### Inicialización de las variables para la simulación

In [48]:
# Creación de la ciudad con estructura deseada
ciudad = Ciudad(
    num_vecindarios=3,
    num_calles_por_vecindario=5,
    num_segmentos_por_calle=4,
    capacidad_segmento=100,
    limite_velocidad=60,
)

# Introducir vehículos al inicio de la simulación
ciudad.introducir_vehiculos(5000)  # Introduce 500 vehículos aleatoriamente en la ciudad

# Creación de una lista para almacenar el número de vehículos en cada segmento después de cada ciclo
historial_vehiculos = []

# Simulación por un número determinado de ciclos
num_ciclos = 100
for _ in range(num_ciclos):
    ciudad.flujo_en_ciudad()
    vehiculos_por_ciclo = []
    for vecindario in ciudad.vecindarios:
        for calle in vecindario.calles:
            for segmento in calle.segmentos:
                vehiculos_por_ciclo.append(segmento.num_vehiculos())
    historial_vehiculos.append(vehiculos_por_ciclo)

### Creacion de Simulación

In [49]:
# Creación de un diccionario para almacenar de forma estructurada los datos
resultados = {}

# Llenamos el diccionario con los resultados
for ciclo in range(num_ciclos):
    resultados[f"Ciclo {ciclo + 1}"] = {}
    idx = 0  # para avanzar por la lista historial_vehiculos
    for v, vecindario in enumerate(ciudad.vecindarios, 1):
        resultados[f"Ciclo {ciclo + 1}"][f"Vecindario {v}"] = {}
        for c, calle in enumerate(vecindario.calles, 1):
            resultados[f"Ciclo {ciclo + 1}"][f"Vecindario {v}"][f"Calle {c}"] = {}
            for s, segmento in enumerate(calle.segmentos, 1):
                resultados[f"Ciclo {ciclo + 1}"][f"Vecindario {v}"][f"Calle {c}"][
                    f"Segmento {s}"
                ] = historial_vehiculos[ciclo][idx]
                idx += 1

# Función para imprimir los resultados de forma estructurada
def imprimir_resultados(resultados):
    for ciclo, datos_ciclo in resultados.items():
        print(f"--- {ciclo} ---")
        for vecindario, datos_vecindario in datos_ciclo.items():
            total_vehiculos_vecindario = sum(
                sum(datos_calle.values()) for datos_calle in datos_vecindario.values()
            )
            print(f"{vecindario} - Total Vehículos: {total_vehiculos_vecindario}")
            for calle, datos_calle in datos_vecindario.items():
                total_vehiculos_calle = sum(datos_calle.values())
                print(f"  {calle} - Total Vehículos: {total_vehiculos_calle}")
                for segmento, num_vehiculos in datos_calle.items():
                    print(f"    {segmento}: {num_vehiculos} vehículos")
        print("\n")


imprimir_resultados(resultados)

--- Ciclo 1 ---
Vecindario 1 - Total Vehículos: 1650
  Calle 1 - Total Vehículos: 329
    Segmento 1: 81 vehículos
    Segmento 2: 75 vehículos
    Segmento 3: 95 vehículos
    Segmento 4: 78 vehículos
  Calle 2 - Total Vehículos: 318
    Segmento 1: 82 vehículos
    Segmento 2: 77 vehículos
    Segmento 3: 80 vehículos
    Segmento 4: 79 vehículos
  Calle 3 - Total Vehículos: 309
    Segmento 1: 72 vehículos
    Segmento 2: 83 vehículos
    Segmento 3: 70 vehículos
    Segmento 4: 84 vehículos
  Calle 4 - Total Vehículos: 343
    Segmento 1: 89 vehículos
    Segmento 2: 79 vehículos
    Segmento 3: 95 vehículos
    Segmento 4: 80 vehículos
  Calle 5 - Total Vehículos: 351
    Segmento 1: 79 vehículos
    Segmento 2: 89 vehículos
    Segmento 3: 90 vehículos
    Segmento 4: 93 vehículos
Vecindario 2 - Total Vehículos: 1653
  Calle 1 - Total Vehículos: 336
    Segmento 1: 98 vehículos
    Segmento 2: 82 vehículos
    Segmento 3: 78 vehículos
    Segmento 4: 78 vehículos
  Calle 2 - Tota

### Función para generar un resumen más limpio de la simulación. 

In [50]:
def generar_resumen(resultados, ciudad):
    primer_ciclo = resultados["Ciclo 1"]
    ultimo_ciclo = resultados[f"Ciclo {num_ciclos}"]

    # Extraer todos los vehículos de la ciudad
    todos_los_vehiculos = [
        vehiculo
        for vecindario in ciudad.vecindarios
        for calle in vecindario.calles
        for segmento in calle.segmentos
        for vehiculo in segmento.vehiculos
    ]

    # Contar los comportamientos
    conteo_comportamientos = {
        "cauteloso": 0,
        "normal": 0,
        "agresivo": 0
    }

    for vehiculo in todos_los_vehiculos:
        conteo_comportamientos[vehiculo.comportamiento] += 1

    total_vehiculos = sum(conteo_comportamientos.values())

    # Número total de vehículos en la ciudad al principio y al final
    total_vehiculos_inicio = sum(
        sum(datos_calle.values())
        for datos_vecindario in primer_ciclo.values()
        for datos_calle in datos_vecindario.values()
    )
    total_vehiculos_final = sum(
        sum(datos_calle.values())
        for datos_vecindario in ultimo_ciclo.values()
        for datos_calle in datos_vecindario.values()
    )

    # Promedio de vehículos por segmento al principio y al final
    total_segmentos = sum(
        len(datos_vecindario) for datos_vecindario in primer_ciclo.values()
    )
    promedio_vehiculos_inicio = total_vehiculos_inicio / total_segmentos
    promedio_vehiculos_final = total_vehiculos_final / total_segmentos

    # Segmentos con mayor y menor cantidad de vehículos al final
    todos_los_segmentos = [
        (vecindario, calle, segmento, vehiculos)
        for vecindario, datos_vecindario in ultimo_ciclo.items()
        for calle, datos_calle in datos_vecindario.items()
        for segmento, vehiculos in datos_calle.items()
    ]
    segmento_mas_congestionado = max(todos_los_segmentos, key=lambda x: x[3])
    segmento_menos_congestionado = min(todos_los_segmentos, key=lambda x: x[3])

    # Imprimir resumen
    print(f"Total de vehículos al inicio: {total_vehiculos_inicio}")
    print(f"Total de vehículos al final: {total_vehiculos_final}")
    print(
        f"Promedio de vehículos por segmento al inicio: {promedio_vehiculos_inicio:.2f}"
    )
    print(
        f"Promedio de vehículos por segmento al final: {promedio_vehiculos_final:.2f}"
    )
    print(
        f"\nSegmento más congestionado al final: {segmento_mas_congestionado[0]} > {segmento_mas_congestionado[1]} > {segmento_mas_congestionado[2]} con {segmento_mas_congestionado[3]} vehículos"
    )
    print(
        f"Segmento menos congestionado al final: {segmento_menos_congestionado[0]} > {segmento_menos_congestionado[1]} > {segmento_menos_congestionado[2]} con {segmento_menos_congestionado[3]} vehículos"
    )

    # Información sobre el comportamiento de los vehículos
    print("\nComportamiento de los vehículos al final de la simulación:")
    for comportamiento, cantidad in conteo_comportamientos.items():
        porcentaje = (cantidad / total_vehiculos) * 100
        print(f"{comportamiento.capitalize()}: {cantidad} vehículos ({porcentaje:.2f}%)")

    print("\nObservaciones:")
    if conteo_comportamientos["agresivo"] > conteo_comportamientos["cauteloso"]:
        print("- Hay más vehículos con comportamiento agresivo que cauteloso. Esto puede haber incrementado la congestión en ciertas áreas.")
    else:
        print("- Hay más vehículos con comportamiento cauteloso que agresivo. Esto puede haber contribuido a un flujo de tráfico más suave en algunas zonas.")
    if conteo_comportamientos["normal"] > total_vehiculos * 0.5:
        print("- La mayoría de los vehículos tienen un comportamiento normal, indicando un equilibrio en la dinámica del tráfico.")


# Llamar a la función de resumen después de la simulación
generar_resumen(resultados, ciudad)

Total de vehículos al inicio: 4960
Total de vehículos al final: 1489
Promedio de vehículos por segmento al inicio: 330.67
Promedio de vehículos por segmento al final: 99.27

Segmento más congestionado al final: Vecindario 3 > Calle 5 > Segmento 2 con 49 vehículos
Segmento menos congestionado al final: Vecindario 1 > Calle 1 > Segmento 1 con 0 vehículos

Comportamiento de los vehículos al final de la simulación:
Cauteloso: 513 vehículos (34.45%)
Normal: 476 vehículos (31.97%)
Agresivo: 500 vehículos (33.58%)

Observaciones:
- Hay más vehículos con comportamiento cauteloso que agresivo. Esto puede haber contribuido a un flujo de tráfico más suave en algunas zonas.


## Respuestas a preguntas

1. ¿Qué es el modelado multiescala y por qué es valioso para comprender el flujo y la congestión del tráfico urbano?

    El modelado multiescala es un enfoque que modela sistemas complejos en diferentes escalas, permitiendo representar tanto los detalles finos (microescala) como el panorama general (macroescala). En el contexto del tráfico urbano, este enfoque es valioso porque la dinámica del tráfico puede ser influenciada por factores a diferentes niveles: desde decisiones individuales de los conductores hasta patrones de tráfico a nivel de ciudad. Al abordar cada escala, el modelado multiescala puede proporcionar una comprensión más detallada y precisa del flujo de tráfico y la congestión.

2. Describe el modelo individual a nivel de vehículo en esta simulación multiescala. ¿Qué parámetros y factores se consideran a esta escala?

    A nivel de vehículo, la simulación se enfoca en el comportamiento individual de cada vehículo. En el modelo:

    - Cada vehículo tiene un *tipo_vehiculo*, aunque en este caso siempre se inicia como "coche".

    - Se define una *velocidad_maxima*.

    - Se registra la *velocidad_actual* del vehículo.

    - El vehículo tiene un comportamiento específico, que puede ser "cauteloso", "normal" o "agresivo". 
    
    Este comportamiento determina cómo acelera o desacelera el vehículo en diferentes circunstancias.

    Los vehículos tienen la capacidad de acelerar (acelerar) o desacelerar (desacelerar) en función de su comportamiento y las condiciones del tráfico.


3. Explique el modelo a nivel de segmento de carretera. ¿Cómo tiene en cuenta la densidad del tráfico, los límites de velocidad y la congestión?

    - Cada segmento de carretera tiene una *capacidad* máxima de vehículos y un *limite_velocidad*.

    - La densidad del tráfico se monitorea a través de la cantidad de vehículos en el segmento (*num_vehiculos*).

    - Si la cantidad de vehículos excede el 80% de la capacidad del segmento, se considera que hay congestion. Cuando hay congestión, los vehículos desaceleran.

    - El flujo de vehículos (flujo_vehiculos) se regula en función de la congestión y la velocidad actual de los vehículos. Si no hay congestión y la velocidad actual del vehículo es menor que el límite, el vehículo puede acelerar. En caso contrario, puede desacelerar.

4. Analice el modelo a nivel de intersección. ¿Qué variables se tienen en cuenta al simular las interacciones de vehículos en las intersecciones?

    - Cada intersección tiene un semáforo (*semaforo*) que puede estar en "rojo" o "verde".

    - Hay un contador (*tiempo_semaforo*) que controla cuándo cambiar el estado del semáforo. En este modelo, el semáforo cambia de estado cada 10 ciclos.

    - Los vehículos que esperan en una intersección se almacenan en *vehiculos_esperando*.

    - Si el semáforo está en verde, y hay vehículos esperando, un vehículo será movido al siguiente segmento (*manejar_interseccion*).
